# Particionado Estratificado del Dataset para Recomendación

*Autora: Lucía Fernández Rodríguez*

*Fecha: 2025*

Este notebook prepara los datos para el entrenamiento y evaluación de un sistema de recomendación a partir de un conjunto de reseñas con embeddings generados previamente.

Se garantiza que cada usuario tenga al menos una reseña en el conjunto de entrenamiento y que todos los restaurantes estén representados en dicho conjunto.

A continuación, se completa la partición siguiendo una proporción estándar del 80% para entrenamiento, 10% para validación y 10% para test. El resultado final se guarda en tres archivos que pueden ser utilizados en modelos de predicción personalizados.

### Importación de librerías

In [ ]:
import pandas as pd
import numpy as np

### Carga del Dataset y Reasignación de identificadores

In [ ]:
# Carga del dataset que ya contiene embeddings de texto e imagen, y otras columnas relevantes
df = pd.read_pickle("/kaggle/input/gijon_embed.pkl")

# Reasignación de IDs: convierte los identificadores de usuario y restaurante a valores numéricos consecutivos
df['user_id_new'] = pd.factorize(df['userId'])[0]
df['restaurant_id_new'] = pd.factorize(df['restaurantId'])[0]

In [ ]:
# Fijar semilla para garantizar reproducibilidad
np.random.seed(10)

# Mezcla aleatoriamente todas las reseñas para evitar sesgos por orden
df = df.sample(frac=1, random_state=10).reset_index(drop=True)

## Selección Inicial: Al Menos una Reseña por Usuario y Restaurante en Train

In [ ]:
# Fase 1: asegurar que cada usuario esté representado al menos una vez en el conjunto de entrenamiento
# Esto evita que haya usuarios en validación o test que el modelo nunca haya visto durante el entrenamiento
train_indices = []
holdout_indices = []

for user_id, group in df.groupby('user_id_new'):
    indices = group.index.tolist()
    if len(indices) == 1:
        # Si el usuario solo tiene una reseña, se asigna directamente a train
        train_indices.extend(indices)
    else:
        # Si tiene varias, se selecciona una al azar para train y el resto para holdout (val/test)
        pick = np.random.choice(indices, 1, replace=False)
        train_indices.extend(pick)
        holdout_indices.extend(list(set(indices) - set(pick)))

train_df = df.loc[train_indices].copy()
holdout_df = df.loc[holdout_indices].copy()

# Fase 2: asegurar que todos los restaurantes estén presentes en el conjunto de entrenamiento
# Esto es importante porque si un restaurante aparece solo en validación/test, el modelo no puede aprender nada sobre él
train_restaurants = set(train_df['restaurant_id_new'].unique())
missing_restaurants = set(holdout_df['restaurant_id_new'].unique()) - train_restaurants

for rest_id in sorted(missing_restaurants):
    candidates = holdout_df[holdout_df['restaurant_id_new'] == rest_id]
    if not candidates.empty:
        # Se mueve una reseña de ese restaurante desde holdout a train
        chosen_idx = np.random.choice(candidates.index, 1, replace=False)
        train_df = pd.concat([train_df, holdout_df.loc[chosen_idx]], ignore_index=True)
        holdout_df = holdout_df.drop(index=chosen_idx)

## División en las tres particiones míticas (80-10-10)

In [ ]:
# Fase 3: aumentar el conjunto de entrenamiento hasta que represente el 80% del total de datos
# Esto se hace para tener una proporción estándar 80-10-10 (train-val-test)
train_target_size = int(0.80 * len(df))
if len(train_df) < train_target_size:
    extra_needed = train_target_size - len(train_df)
    extra_samples = holdout_df.sample(n=extra_needed, random_state=10)
    train_df = pd.concat([train_df, extra_samples], ignore_index=True)
    holdout_df = holdout_df.drop(index=extra_samples.index)

# Fase 4: dividir el 20% restante entre validación y test (10% cada uno)
# Este conjunto ya se ha mezclado aleatoriamente, así que lo partimos por la mitad
holdout_df = holdout_df.sample(frac=1, random_state=10).reset_index(drop=True)
val_size = int(0.5 * len(holdout_df))
val_df = holdout_df.iloc[:val_size].copy()
test_df = holdout_df.iloc[val_size:].copy()

### Selección de columnas y guardado de archivos

In [ ]:
# Selección de columnas relevantes para el análisis o el entrenamiento posterior
cols_to_keep = ['images', 'rating', 'restaurantId', 'reviewId', 'text', 'userId',
                'sel_image_url', 'image_emb', 'text_emb']

train_df = train_df[cols_to_keep + ['user_id_new', 'restaurant_id_new']]
val_df   = val_df[cols_to_keep + ['user_id_new', 'restaurant_id_new']]
test_df  = test_df[cols_to_keep + ['user_id_new', 'restaurant_id_new']]

# Guardado de los datasets para uso posterior
train_df.to_pickle("/kaggle/working/train_v80.pkl")
val_df.to_pickle("/kaggle/working/val_v10.pkl")
test_df.to_pickle("/kaggle/working/test_v10.pkl")

# Verificación del resultado: tamaños y proporciones
print("\n Datos guardados con éxito.")
print(f"Train: {train_df.shape}")
print(f"Val:   {val_df.shape}")
print(f"Test:  {test_df.shape}")

total = len(df)
print(f"\n Porcentajes aproximados:")
print(f"Train: {len(train_df)/total:.2%}")
print(f"Val:   {len(val_df)/total:.2%}")
print(f"Test:  {len(test_df)/total:.2%}")